# Orbit Propagation Demo 

This tutorial demonstrates how to perform orbit propagation through numerical integration.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.display import create_2d_map, create_3d_globe
from ostk.astrodynamics.flight.system import Dynamics, SatelliteSystem
from ostk.astrodynamics.flight.system.dynamics import (
    AtmosphericDrag,
    CentralBodyGravity,
    PositionDerivative,
    ThirdBodyGravity,
)
from ostk.astrodynamics.trajectory import Orbit, Propagator, State
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.utilities import convert_state
from ostk.core.filesystem import Directory
from ostk.mathematics.geometry.d3.objects import Composite, Cuboid, Point
from ostk.physics import Environment
from ostk.physics.coordinate import Frame, Position, Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Interval, Scale
from ostk.physics.units import Mass

## Simulation Setup

### **Define the physical `Environment`**

In [ ]:
instant_J2000 = Instant.J2000()
celestial_objects = [
    Earth.EGM96(360, 360),  # Earth.EGM2008(), Earth.WGS84_EGM96(), Earth.EGM84(180, 180)
    Sun.default(),
    Moon.default(),
]

environment = Environment(instant_J2000, celestial_objects)

### **Define the `SatelliteSystem` with properties:**
* `Mass`
* `Geometry`
* `Inertia Tensor`
* `Surface Area`
* `Drag Coefficient`

In [ ]:
mass = Mass(90.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(
    Cuboid(
        Point(0.0, 0.0, 0.0),
        [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
        [1.0, 0.0, 0.0],
    )
)
inertia_tensor = np.identity(3)
surface_area = 0.8
drag_coefficient = 2.2

satellite_system = SatelliteSystem(
    mass, satellite_geometry, inertia_tensor, surface_area, drag_coefficient
)

### **Define the initial `State` of the `SatelliteSystem`:**
* ` start_instant `
* ` start_position `
* ` start_velocity `

In [ ]:
start_instant = Instant.date_time(DateTime.parse("2023-03-20T00:00:00.000"), Scale.UTC)

start_position = Position.meters(
    [-1514668.9408102570269, -192084.12149140036718, 6831711.4584368728174],
    Frame.GCRF(),
)
start_velocity = Velocity.meters_per_second(
    [-6348.0791876050259859, 3867.5824926981121621, -1297.1761044290490705],
    Frame.GCRF(),
)

start_state = State(start_instant, start_position, start_velocity)

### **Define the Dynamics to consider for Propagation**
* Central Body Gravity
* Third Body Gravity
* Atmospheric Drag
* (Soon) Thruster Dynamics

In [ ]:
earth = Earth.from_models(
    EarthGravitationalModel(
        EarthGravitationalModel.Type.EGM2008,
        Directory.undefined(),
        20,
        20,
    ),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Exponential),
)
central_body_gravity = CentralBodyGravity(earth)
sun_third_body_gravity = ThirdBodyGravity(Sun.default())
moon_third_body_gravity = ThirdBodyGravity(Moon.default())
atmospheric_drag = AtmosphericDrag(earth, satellite_system)

In [ ]:
dynamics_list = [
    central_body_gravity,
    sun_third_body_gravity,
    moon_third_body_gravity,
    atmospheric_drag,
    PositionDerivative(),  # Required by default
]

### **Define a `NumericalSolver` (leveraging `boost::numeric::odeint`) for numerical integration with:**
* `NumericalSolver.LogType`
* `NumericalSolver.StepperType`
* `TimeStep`
* `RelativeTolerance`
* `AbsoluteTolerance`

Alternatively, you can use `NumericalSolver.default()`

In [ ]:
numerical_solver = NumericalSolver(
    NumericalSolver.LogType.NoLog,
    NumericalSolver.StepperType.RungeKuttaFehlberg78,
    5.0,
    1.0e-15,
    1.0e-15,
)

### **Define a `Propagator`**

In [ ]:
propagator = Propagator(numerical_solver, dynamics_list)

## Propagate

Define the propagation `instants` of interest

In [ ]:
instants = Interval.closed(
    start_state.get_instant(), start_state.get_instant() + Duration.minutes(15.0)
).generate_grid(Duration.seconds(15.0))

Propagate from `start_state` for all `instants` defined

In [ ]:
states = propagator.calculate_states_at(start_state, instants)

Format the output data

In [ ]:
data = [convert_state(state.get_instant(), state) for state in states]

Compute classical orbital elements for each state propagated

In [ ]:
for i, state in enumerate(states):
    coe = COE.cartesian(
        (state.get_position(), state.get_velocity()),
        earth.get_gravitational_parameter(),
    )
    data[i] += [
        coe.get_semi_major_axis().in_kilometers(),
        coe.get_raan().in_degrees(),
    ]

In [ ]:
orbit_df = pd.DataFrame(
    data,
    columns=[
        "$Time^{UTC}$",
        "$MJD^{UTC}$",
        "$x_{x}^{ECI}$",
        "$x_{y}^{ECI}$",
        "$x_{z}^{ECI}$",
        "$v_{x}^{ECI}$",
        "$v_{y}^{ECI}$",
        "$v_{z}^{ECI}$",
        "$Latitude$",
        "$Longitude$",
        "$Altitude$",
        "$SemiMajorAxisKm$",
        "$RaanDegrees$",
    ],
)

## Display

In [ ]:
orbit_df.head()

2D plot, over **World Map**:

In [ ]:
figure = create_2d_map(
    data=go.Scattergeo(
        lon=orbit_df["$Longitude$"],
        lat=orbit_df["$Latitude$"],
        mode="lines",
        line=go.scattergeo.Line(width=1, color="red"),
    ),
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = create_3d_globe(
    data=[
        go.Scattergeo(
            lon=orbit_df["$Longitude$"],
            lat=orbit_df["$Latitude$"],
            mode="lines",
            line=go.scattergeo.Line(width=2, color="rgb(255, 62, 79)"),
        )
    ],
)

figure.show()